In [1]:
import os
import tempfile
import functools

import jax
import jax.numpy as jnp

from absl import logging

import flax
import optax
import chex
from clu import metric_writers, parameter_overview

from molnet import train_state, hooks, train, utils, loss
from molnet.models import create_model

from molnet.data import input_pipeline
from configs.tests import unet_test

from typing import Any, Dict, Tuple

2024-11-04 15:16:56.196705: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
config = unet_test.get_config()
config.root_dir = '/l/data/molnet/atom_maps'
config.workdir = tempfile.mkdtemp()

# Create writer for logs
writer = metric_writers.create_default_writer(config.workdir)
writer.write_hparams(config.to_dict())


2024-11-04 15:15:01.057940: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [3]:
# Get datasets
rng = jax.random.PRNGKey(config.rng_seed)
rng, data_rng = jax.random.split(rng)
datasets = input_pipeline.get_datasets(data_rng, config)
train_ds = datasets["train"]


[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:1: Invalid control characters encountered in text.
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:3: Expected identifier, got: 12126270835042841805


In [4]:
x_init = next(train_ds)['images']
rng, init_rng = jax.random.split(rng)
model = create_model(config)

variables = model.init(init_rng, x_init, training=True)
params = variables["params"]
batch_stats = variables["batch_stats"]
parameter_overview.log_parameter_overview(params)

# Create optimizer
tx = utils.create_optimizer(config)

# Create training state
state = train_state.TrainState.create(
    apply_fn=model.apply,
    params=params,
    tx=tx,
    batch_stats=batch_stats,
    best_params=params,
    step_for_best_params=0,
    metrics_for_best_params={},
    train_metrics=train.Metrics.empty(),
)

[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:1: Invalid control characters encountered in text.
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:3: Expected identifier, got: 12126270835042841805
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:1: Invalid control characters encountered in text.
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:3: Expected identifier, got: 12126270835042841805
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing 

In [5]:
# Set up checkpointing
checkpoint_path = os.path.join(config.workdir, "checkpoint")
checkpoint_hook = hooks.CheckpointHook(
    checkpoint_path, max_keep=1
)
state = checkpoint_hook.restore_or_init(state)
initial_step = state.get_step()

train_metrics_hook = hooks.LogTrainMetricsHook(
    writer,
)
evaluate_model_hook = hooks.EvaluateModelHook(
    evaluate_model_fn=lambda state: train.evaluate_model(
        state,
        datasets,
        config.num_eval_steps,
    ),
    writer=writer,
)

In [9]:
state = evaluate_model_hook(state)

(16, 128, 128, 10, 1)
(16, 128, 128, 21, 5)
(16, 128, 128, 10, 1)
(16, 128, 128, 21, 5)
(16, 128, 128, 10, 1)
(16, 128, 128, 21, 5)
(16, 128, 128, 10, 1)
(16, 128, 128, 21, 5)


[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:1: Invalid control characters encountered in text.
[libprotobuf ERROR external/com_google_protobuf/src/google/protobuf/text_format.cc:337] Error parsing text-format tensorflow.data.experimental.DistributedSnapshotMetadata: 1:3: Expected identifier, got: 2460497826159879776
